In [ ]:
%%capture installation_results
def install_packages():
    ! pip install geopy
    ! pip install geopandas
    ! pip install folium
install_packages()

In [ ]:
import altair as alt 
import geopy
import geopandas

import pandas as pd
import folium

import tensorflow as tf
import numpy as np
import os
import re
import dill
import base64
import requests
import PIL 
import matplotlib.pyplot as plt
import gc
import tarfile
import os.path
import shutil

from time import sleep 
from PIL import Image
from matplotlib.pyplot import Figure
from shutil import rmtree
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from tensorflow.image import resize
from tensorflow.data import Dataset 
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array, smart_resize,array_to_img,\
    ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.efficientnet import EfficientNetB1, EfficientNetB2, EfficientNetB3,\
    EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
from tensorflow.data.experimental import save, cardinality
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.backend import clear_session
from tensorflow.distribute import get_strategy
from tensorflow.keras.models import load_model
from itertools import chain
from geopy import Nominatim
from folium import IFrame
from geopy.extra.rate_limiter import RateLimiter 
from pandas.io.json import read_json 

In [ ]:
! pip freeze

In [ ]:
pd.set_option('display.max_rows', 400)

MAIN_STORAGE="/home/jovyan/jhshi/capstonedump"
PHOTOS_SOURCE=f"{MAIN_STORAGE}/photos"
LATENT_VECTORS_STORAGE=f"{MAIN_STORAGE}/latents"
TFMODELS_STORAGE=f"{MAIN_STORAGE}/tfmodels"
PLOTS_STORAGE=f"{MAIN_STORAGE}/plots"
LISTINGS_STORAGE=f"/home/jovyan/jhshi/capstoneapp/downloads"
APP_DUMPATH="/home/jovyan/jhshi/capstoneapp"
CATEGORY="kitchen"
BATCH=(20, 30, 50)
IMG_SIZE={
    "EfficientNetB0":224,
    "EfficientNetB1":240,
    "EfficientNetB2":260,
    "EfficientNetB3":300,
    "EfficientNetB4":380,
    "EfficientNetB5":456,
    "EfficientNetB6":528,
    "EfficientNetB7":600, 
    "InceptionV3":299
} 
LATENT_EPOCHS=1
N_CLASSES=3
N_COLORS=3
STRATEGY=get_strategy() 
OPTIMIZATION_SUBDATASETS=("training", "validation")
MODEL_SPEC="EfficientNetB5"
LOCATOR=Nominatim(user_agent="myGeoendoer")

In [ ]:
! tar -xvf addresses-and-images.tar.gz -C /home/jovyan/jhshi/capstoneapp

## train some images TF

In [ ]:
def split_factory(**KWARGS):
    return image_dataset_from_directory(
                KWARGS["image_source"], 
                labels="inferred", 
                label_mode="int",
                class_names=list(map(str,  
                                     range(N_CLASSES))),
                color_mode="rgb",
                image_size=(KWARGS['image_size'],  
                            KWARGS['image_size']),
                subset=KWARGS["subset"],
                shuffle=True,
                batch_size=KWARGS['batch_size'],
                seed=1,
                validation_split=0.20)

def pretrained_factory(**KWARGS):
    clear_session()
    expert=EfficientNetB5(
                include_top=True, 
                input_shape=(KWARGS["img_size"],  
                             KWARGS["img_size"],  
                             N_COLORS))
    pretrained=tf.keras.Model([expert.input],
                   [expert.layers[-KWARGS["num_layers_to_exclude"]] 
                          .output])
    pretrained.trainable=False
    pipeline=Sequential()
    pipeline.add(RandomFlip('horizontal', 
                            seed=1,
                            input_shape=(KWARGS['img_size'], 
                                         KWARGS['img_size'], 
                                         N_COLORS)))
    pipeline.add(RandomRotation(KWARGS['rotation_factor'], 
                                seed=1))
    pipeline.add(RandomContrast(KWARGS['contrast_factor'], 
                                seed=1))
    pipeline.add(pretrained)
    pipeline.summary()
    return pipeline

def load_latent_vectors(in_load_path, 
                        **KWARGS
                       ):
    unsorted=[(int(re.split("\-|\.", file)[-2]), 
               np.load(f'{in_load_path}/{file}'))
              for file 
              in os.listdir(in_load_path)]
    ordered=sorted(unsorted,  
                   key = lambda index_dataset: 
                             index_dataset[0])
    
    stacked=(np.vstack(
                [dataset  
                 for index, dataset  
                 in ordered]))
    return stacked 

def one_hot_labels(in_label_images):
    """
    :in_label_images: 
        BatchDataset: image arrays (batch, pixels, pixels, colors) 
            && labels
    """
    unpacked_=list(
                 chain(
                     *[labels 
                       for images, labels 
                       in in_label_images])) 
    return tf.one_hot(unpacked_,  
                      depth=N_CLASSES)

def clean_directories_in(in_dir, **KWARGS):
    if in_dir:
        rmtree(in_dir)
    os.mkdir(in_dir)
    [os.mkdir(f"{in_dir}/{subdirectory}")  
         for subdirectory 
         in KWARGS['subdir'] ]

def classifier_facotory(**KWARGS):
    classifier=Sequential()
    classifier.add(
        Dense(
            N_CLASSES, 
            name="softmax",
            activation="softmax",
            input_dim=KWARGS['input_dim']))

    classifier.compile(
        optimizer=SGD(lr=0.0001,  
                      momentum=1.,  
                      nesterov=True),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return classifier

def clean_directories_in(in_dir, **KWARGS):
    if in_dir:
        rmtree(in_dir)
    os.mkdir(in_dir)
    [os.mkdir(f"{in_dir}/{subdirectory}")  
         for subdirectory 
         in KWARGS['subdir'] ]

def calculate_latents(in_distributed_dataset, 
                      in_pretrained_model, 
                      **KWARGS
                     ): 
    SAVEPATH_="{}/{}/vectors-per-step-{}.npy"
    with STRATEGY.scope():
        for subset, dataset in in_distributed_dataset.items():
            for _ in range(LATENT_EPOCHS):
                for index, bundle in enumerate(
                                         iter(
                                             dataset)):
                    try:
                        latent_per_step=(in_pretrained_model 
                                         .predict(
                                             tf.convert_to_tensor(bundle[0]), 
                                             verbose=0))
                        clear_session()
                        gc.collect()
                    except tf.errors.InvalidArgumentError: 
                        pass
                    with open(SAVEPATH_.format(KWARGS["latents_save_location"],
                                               subset, 
                                               index),  
                              "wb") \
                        as fout_:
                        np.save(fout_,
                                latent_per_step,
                                allow_pickle=True) 
                    del latent_per_step

def compress_latents(in_path, out_path):
    with tarfile.open(out_path, "w:gz") as tar_:
        tar_.add(in_path, 
                 arcname=os.path.basename(in_path))

In [ ]:
split={subset: 
           split_factory(image_source=PHOTOS_SOURCE,
                         subset=subset, 
                         image_size=IMG_SIZE[MODEL_SPEC], 
                         batch_size=BATCH[0]) 
       for subset  
       in OPTIMIZATION_SUBDATASETS} 

distributed={index: 
             STRATEGY.experimental_distribute_dataset(dataset)
             for index, dataset in split.items()}

transfer_student=pretrained_factory(
                     img_size=IMG_SIZE[MODEL_SPEC],
                     num_layers_to_exclude=2,
                     rotation_factor=0.1,
                     contrast_factor=0.1)

In [ ]:
try:
    latents_and_labels
    del latents_and_labels
    gc.collect()
except NameError:
    pass 

latents_and_labels={subset:
                    (load_latent_vectors(f"{LATENT_VECTORS_STORAGE}/{subset}/"),
                     one_hot_labels(split[subset]))
                         for subset in OPTIMIZATION_SUBDATASETS}
latent_dimension=latents_and_labels["training"][0].shape[1] 

In [ ]:
classifier=load_model(f"{TFMODELS_STORAGE}/{MODEL_SPEC}")

In [ ]:
all_listings=[f"{LISTINGS_STORAGE}/{file}"  
              for file 
              in os.listdir(LISTINGS_STORAGE) 
              if file.endswith((".jpeg", ".jpg"))]

In [ ]:
%%time 
def get_index_tags(in_image):
    """
    :in_image: jpg or jpeg 
    note name of the image reflects the indexing in the listing imageset
    predicts quality from the in_image
    """
    rescaled= resize(img_to_array(Image.open(in_image)), 
                     [IMG_SIZE[MODEL_SPEC], IMG_SIZE[MODEL_SPEC]])
    index=re.split(r"\/|\.", in_image)[-2]
    reshaped=tf.convert_to_tensor(np.array([rescaled,]))
    latent_vector=transfer_student.predict(reshaped, verbose=0)
    tag=np.argmax(classifier.predict(latent_vector), axis=1)[0]
    clear_session()
    return (int(index), tag)
    
index_tags=[get_index_tags(file) for file in all_listings]
ordered=sorted(index_tags, key=lambda x: x[0])
indiced_tags=[pair[1] for pair in ordered]

In [ ]:
try:
    os.remove(f"{APP_DUMPATH}/index_and_tags.pkd")
except FileNotFoundError:
    pass
with open(f"{APP_DUMPATH}/index_and_tags.pkd", "wb") as writefile:
    dill.dump(indiced_tags, writefile)

## altair a single property

In [ ]:
property_address="301 Van Ness Ave, San Francisco, CA 94102"
locator=Nominatim(user_agent="myGeoendoer")
location=locator.geocode(property_address)

In [ ]:
address_coordinates=pd.DataFrame(
    {
        "address": [property_address],
        "longitude":[location.longitude],
        "latitude":[location.latitude]
    }
)

In [ ]:
displayer=folium.Map(
                    location=[location.latitude, 
                              location.longitude], 
                    zoom_start=2)

In [ ]:
folium.Marker(
    location=[location.latitude, 
              location.longitude],
    popup=property_address
).add_to(displayer)

In [ ]:
displayer

## altaïr property listings

In [ ]:
def preload_listing_profiles(**KWARGS):
    with open(KWARGS["in_addresses_loadpath"], "rb") as readfile:
        indices_and_addresses=dill.load(readfile)
    sorted_addresses=[addr for index, addr in indices_and_addresses]
    images=["{}/{}".format(KWARGS["in_images_loadpath"], file)
            for file 
            in os.listdir(KWARGS["in_images_loadpath"])
            if file.endswith((".jpeg", ".jpg"))]
    sorted_images=sorted(
        images,
        key=
        lambda x:
        int(re.split(r"\/|\.", x)[-2])
    )
    with open(KWARGS["in_tags_loadpath"], "rb") as readfile:
        indiced_tags=dill.load(readfile)
    return {
        "out_sorted_tags":indiced_tags,
        "out_sorted_images":sorted_images,
        "out_sorted_addresses":sorted_addresses
    }

def make_listing_profiles(**KWARGS):
    return pd.DataFrame(
        {
            "tags":KWARGS["in_sorted_tags"],
            "images": KWARGS["in_sorted_images"],
            "addresses":KWARGS["in_sorted_addresses"]
        }
    )

preloaded=preload_listing_profiles(
            in_addresses_loadpath=f"{LISTINGS_STORAGE}/address_index.pkd" ,
            in_tags_loadpath=f"{APP_DUMPATH}/index_and_tags.pkd",
            in_images_loadpath=LISTINGS_STORAGE)

profile_with_null_addresses=make_listing_profiles(
            in_sorted_tags=preloaded["out_sorted_tags"],
            in_sorted_images=preloaded["out_sorted_images"],
            in_sorted_addresses=preloaded["out_sorted_addresses"])

In [ ]:
with open(f"{APP_DUMPATH}/listing_profiles.pkd", "wb") as writefile:
    dill.dump(profile, writefile)

In [ ]:
POPUP_HTML_SCRIPT='''
    <img src="data:image/png;base64,{}">
    <p> address: {} </p>
    <p> room quality: {} </p>
'''
ROOM_INDEX_TO_QUALITY={
    0:"poor",
    1:"normal",
    2:"luxurious"
}
POPUP_FRAME_SIZE=300
def make_popup(**KWARGS):
    with open(KWARGS["in_image_source"], "rb") as readfile:
        encoded_image=base64.b64encode(readfile.read())
        image_tag=POPUP_HTML_SCRIPT.format
        in_frame_=IFrame(image_tag(encoded_image.decode('UTF-8'),
                                  KWARGS["in_location"],
                                  ROOM_INDEX_TO_QUALITY[KWARGS["in_tag"]]
                                 ),
                        width=POPUP_FRAME_SIZE, height=POPUP_FRAME_SIZE)
        return folium.Popup(in_frame_, max_width=POPUP_FRAME_SIZE)

In [ ]:
def mark_location_on_map(in_map, **KWARGS):
    marker_popup=make_popup(
                    in_location=KWARGS["in_location"],
                    in_tag=KWARGS["in_tag"],
                    in_image_source=KWARGS["in_image_source"])
    folium.Marker(location= KWARGS["in_coordinates"],
                  popup=marker_popup
                 ).add_to(in_map)

In [ ]:
%%capture profile_printout
def trim_address(in_address):
    split=re.split(r"\,|MLS",in_address)
    trim_street_ends=re.sub(r"APT\s+[0-9]+|[\/|\#][\s,a-z,A-Z,0-9]+|[UNIT|SPACE]\s[0-9]+", "", split[0])
    return f"{trim_street_ends},CA"

profile=profile_with_null_addresses[~profile_with_null_addresses
                                    .addresses
                                    .isna()]

num_address_tokens=(profile
 .addresses
 .apply(
     lambda x: re.split(r"\,|MLS", x))
 .str.len())

profile_valid_addresses=profile[num_address_tokens>2] 

profile_valid_addresses['truncated_addresses']=profile_valid_addresses.addresses.apply(
    lambda x: trim_address(x)
)

profile_valid_addresses.drop(
    profile_valid_addresses[
        profile_valid_addresses.truncated_addresses=="171 Beth Eden St,CA"].index) 

In [ ]:
%%time
def locate_coordinate(in_address):
    sleep(1)
    coordinates_=geocode(in_address) 
    if coordinates_:
        return (in_address, 
                (coordinates_[0], coordinates_[1])
               )
    else:
        print(f"bad address: {in_address}")
geocode = RateLimiter(LOCATOR.geocode, 
                      min_delay_seconds=1)
coordinates=[locate_coordinate(line) 
             for line 
             in list(profile_valid_addresses.truncated_addresses)]
coordinates=[file for file in coordinates if file] 
coordinates_with_addresses=pd.DataFrame(coordinates).rename({0:"truncated_addresses", 
                                  1:"address_coordinates"}, 
                                 axis=1)

In [ ]:
with open(f"{APP_DUMPATH}/geo-coordinates-listings.pkd", "wb") as writefile:
    dill.dump(coordinates, writefile)

In [ ]:
with open(f"{APP_DUMPATH}/geo-coordinates-listings.pkd", "rb") as readfile:
    coordinates = dill.load(readfile)

In [ ]:
inspections=list(coordinates_with_addresses.truncated_addresses)
profile_valid_addresses=profile_valid_addresses[
    profile_valid_addresses.truncated_addresses.isin(inspections)]

profile_valid_coordinates= (
    pd
    .merge(
        coordinates_with_addresses,
        profile_valid_addresses,
        on="truncated_addresses"))

In [ ]:
profile_valid_coordinates["coordinates"]=(profile_valid_coordinates 
                                          .address_coordinates 
                                          .apply(lambda x: x[1]))

In [ ]:
def clean_and_derive_coordinates(in_profile):
    profile=in_profile[~in_profile
                       .addresses
                       .isna()]

    num_address_tokens=(profile
     .addresses
     .apply(
         lambda x: re.split(r"\,|MLS", x))
     .str.len())

    profile_valid_addresses=profile[num_address_tokens>2] 

    profile_valid_addresses['truncated_addresses']=(profile_valid_addresses 
                                                    .addresses.apply(
                                                        lambda x: trim_address(x)))

    (profile_valid_addresses
     .drop(
         profile_valid_addresses[
             profile_valid_addresses
             .truncated_addresses=="171 Beth Eden St,CA"]
         .index))

    geocode = RateLimiter(
                  LOCATOR.geocode, 
                  min_delay_seconds=1)
    
    coordinates=[locate_coordinate(line) 
                 for line 
                 in list(profile_valid_addresses.truncated_addresses)]
    coordinates=[file for file in coordinates if file] 
    coordinates_with_addresses=pd.DataFrame(coordinates).rename({0:"truncated_addresses", 
                                      1:"address_coordinates"}, 
                                     axis=1)
    inspections=list(coordinates_with_addresses.truncated_addresses)
    profile_valid_addresses=profile_valid_addresses[
        profile_valid_addresses.truncated_addresses.isin(inspections)]

    profile_valid_coordinates= (
        pd
        .merge(
            coordinates_with_addresses,
            profile_valid_addresses,
            on="truncated_addresses"))

    profile_valid_coordinates["coordinates"]=(profile_valid_coordinates 
                                              .address_coordinates 
                                              .apply(lambda x: x[1]))
    return profile_valid_coordinates

In [ ]:
clean_and_derive_coordinates()

In [ ]:
with open(f"{APP_DUMPATH}/listing-profile-with-tags-coordinates-and-addresses", "wb") as writefile:
    dill.dump(profile_valid_coordinates.to_json(), writefile)

In [ ]:
with open(f"{APP_DUMPATH}/listing-profile-with-tags-coordinates-and-addresses", "rb") as readfile:
    profile_valid_coordinates=read_json(dill.load(readfile))

In [ ]:
def plot_map_with_house_listings(in_property_profile):
    SF_CITYHALL="301 Van Ness Ave, San Francisco, CA 94102"
    displayer_=folium.Map(location=list(LOCATOR.geocode(SF_CITYHALL)[1]),
                          zoom_start=6)

    (in_property_profile
     .apply(lambda x: 
            mark_location_on_map(
                displayer_,
                in_location=x["address_coordinates"][0],
                in_tag=x["tags"],
                in_image_source=x["images"],
                in_coordinates=x["coordinates"]), 
            axis=1))
    return displayer_.json
map_display=plot_map_with_house_listings(profile_valid_coordinates)

In [ ]:
map_display

In [ ]:
SF_CITYHALL="301 Van Ness Ave, San Francisco, CA 94102"
LOCATOR.geocode(SF_CITYHALL)[1]